# Generating basic waveforms #

A notebook originally built to solve exercise 5 of the second chapter of the ThinkDSP book. This notebook creates basic waveforms such as sawtooth, square and triangle but also plays a bit further with the phase and harmonics.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import ipywidgets as widgets

### Widgets for GUI ###

In [3]:
comboHarm = widgets.Dropdown(
    options=[('Even', 1), ('Odd', 2), ('Even & Odd', 3)],
    value=3,
    description='Harmonics:',
)

comboPhase = widgets.Dropdown(
    options=[('Constant', 1), ('Random', 2)],
    value=1,
    description='Phase:',
)

sPhase = widgets.FloatSlider(
    value=np.pi/2,
    min=0,
    max=np.pi,
    step=np.pi/4,
    description='Phase Constant:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

comboDec = widgets.Dropdown(
    options=[('1/f', 1), ('1/f^2', 2)],
    value=2,
    description='Decrease:',
)

### Global Settings ###

In [2]:
fs = 48e3 # Samplerate
f = 100 # Frequency of fundamental
T = 100 / f # Time duration
N = int(T * fs) # Duration in samples
t = np.arange(N) / fs # time vector

### Function to generate waveforms with GUI ###

In [4]:
x = 0 # Variable to store signal
i = 0 # Iterator for harmonics

def createWave(harmType, dec, phi, kPhi):
    '''
        createWave is a function for generating
        a waveform by choosing the type of harmonics
        the decrease of the harmonics and the phase
        of the harmonics. the function modifies the
        value of global variable x to be the wave
        generated and displays the waveform and
        spectrum.

        - harmType: 
            * Sets the type of harmonics generated.
                1 => Even
                2 => Odd
                3 => Odd & Even
        - dec:
            * Sets the amplitude decrease of the harmonics.
                1 => 1/f
                2 => 1/f**2
        - phi:
            * Sets the phase to be a constant value or random.
                1 => constant
                2 => random
        - kPhi:
            * Sets the constant value for the phase.
            * Floating number that varies from 0 to np.pi
    '''

    global x, i
    
     # Harmonic content of signal (bandwidth limited to Nyquist)
    if harmType == 1: # If even
        i = np.concatenate(([f], np.arange(f*2, int(fs/2), 2*f)))
    elif harmType == 2: # If odd
        i = np.arange(f, int(fs/2), 2*f)
    elif harmType == 3: # if even & odd
        i = np.arange(f, int(fs/2), f)

    x = np.zeros_like(t) # Variable to store signal
    
    # choose phase
    if phi == 1: # If constant
        phase = np.ones(len(i)) * kPhi
    else: # If random
        phase = np.random.randn(len(i))
        phase = phase / np.max( np.abs(phase) ) * 2 * np.pi
    
    # choose harmonic decrease
    if dec == 1: # If 1/f
        decrease = 1
    else: # If 1/f^2
        decrease = 2
        
    # Generate periodic signal
    for n in range(len(i)):
        x += np.sin(2 * np.pi * t * i[n] + phase[n]) / (i[n]/f)**decrease
        
    x = x / np.max( np.abs(x) ) # normalize
    
    # Plot waveform    
    %matplotlib inline
    plt.figure(figsize=(6,4))
    plt.subplot(2,1,1)
    plt.title("Waveform of synthesized signal")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude [V]")
    plt.grid()
    plt.plot(t, x, color='navy')
    plt.xlim((0,4/f))
    plt.tight_layout()

    X = np.fft.rfft(x) / fs # Compute FFT (no window needed as the duration set corresponds with the period)
    freq = np.fft.rfftfreq(len(x), 1/fs) # Create frequency vector

    # Plot Spectrum
    plt.subplot(2,1,2)
    plt.title("Spectrum of synthesized signal")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Magnitude [dB]")
    plt.grid()
    plt.semilogx(freq, 20 * np.log10(np.abs(X)), color='navy')
    plt.xlim((10,fs/2))
    plt.ylim((-150,0))
    plt.tight_layout()
    plt.show()

widgets.interact_manual(createWave, harmType=comboHarm ,dec=comboDec, phi=comboPhase, kPhi=sPhase) 

interactive(children=(Dropdown(description='Harmonics:', index=2, options=(('Even', 1), ('Odd', 2), ('Even & O…

<function __main__.createWave(harmType, dec, phi, kPhi)>

Run the following cell after clicking 'Run Interact'. Try listening to the signal by applying a fixed phase or a random phase to hear if phase causes any audible change to the waveform. Careful, the audio could be loud or contain a DC component!

In [29]:
ipd.Audio(x * np.hanning(len(x)), rate=fs) # window to avoid clicks